### Reinforcement Learning: Theory and Algorithms

ここでは、第2章における各定理、補題、系などをまとめます。証明は初見でもこれを参考にすれば導出できるように丁寧に記します。

$(s,a)$を受け取り、$s^{\prime} \sim P(\cdot \mid s, a)$と報酬$r(s,a)$を出力する**生成モデル**を考えます。生成モデルを用いて達成したいのは$P$をサンプルから経験的に推定し、プランニングを解くことでほぼ最適(near-optimal)な方策を獲得することです。この章において関心のあることは、**Near-Optimalな方策を獲得するために必要な生成モデルからのサンプル数は最低どれくらいか？(worsr-caseでの最低サンプル数)** です。

### 定理2.1 
* $P$を推定しますが、$P$は$|\mathcal{S}|^2|\mathcal{A}|$個パラメータをもった行列ですので、計算量で考えると、$|\mathcal{S}|^2|\mathcal{A}|$このペアを$N$回サンプルすればよさそうです。計算量で示すと$O\left(|\mathcal{S}|^2|\mathcal{A}|\right)$となります。$N$がないのは、定数であるため、計算量に対する影響が少ないためだと思います。

* 生成モデルから$|\mathcal{S}||\mathcal{A}| N $回サンプルするとして、いい感じに$P$を推定したいわけです。**いい感じとは、具体的にはどれくらい**のことなのでしょうか？そして、**いい感じに推定するために必要なサンプル数**は最低でもどれくらい必要でしょうか？

* 命題2.1では仮定として、**いい感じに推定するために必要なサンプル数**を示しており、その後に**いい感じ**の具体的な定式化をしています。

#### 仮定
$$
\text {生成モデルからのサンプル数} =|\mathcal{S}||\mathcal{A}| N \geq \frac{\gamma}{(1-\gamma)^4} \frac{|\mathcal{S}|^2|\mathcal{A}| \log (c|\mathcal{S}||\mathcal{A}| / \delta)}{\epsilon^2}
$$

かつ

$\epsilon \in\left(0, \frac{1}{1-\gamma}\right)$

を満たすような定数$c$が存在すると仮定します。

#### 命題2.1

- （モデル精度）遷移モデルの誤差は、次のように制限されます。

$$
\max _{s, a}\|P(\cdot \mid s, a)-\widehat{P}(\cdot \mid s, a)\|_1 \leq(1-\gamma)^2 \epsilon .
$$

- （一様値精度）すべての方策$\pi$に対して、

$$
\left\|Q^\pi-\widehat{Q}^\pi\right\|_{\infty} \leq \epsilon
$$

- （Near optimalなPlanning）$\widehat{\pi}$が$\widehat{M}$における最適な方策であると仮定します。次のことが成り立ちます。

$$
\left\|\widehat{Q}^{\star}-Q^{\star}\right\|_{\infty} \leq \epsilon, \text { and }\left\|Q^{\widehat{\pi}}-Q^{\star}\right\|_{\infty} \leq 2 \epsilon .
$$

* モデル精度について：仮定のサンプル数をこなせば、$P$の推定誤差の上階を得ることができます。$\gamma$が大きいほど値は小さくなります。(**Why**)
* 一様値精度について：真のMDPでの任意の方策を、推定MDPで実行したときの誤差の上界を示しています。$\epsilon$以外には依存しませんが、これはいい感じに仮定を選んでいるからだと思います。
* Near optimalなPlanningについて：推定MDPにおける最適なQ関数を推定MDPで実行するか、真のMDPで実行するかを考えると、前者の値が高く、後者がひくくなります。

##### サンプル数下界の各パラメータを動かして、どのように値が変わるか見てみましょう

In [7]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Given values
S = 2
A = 2
gamma = 0.9  
c = 1  
delta = 0.05  
epsilon = 0.1 
# 生成モデルからのサンプル数
N = (gamma / (1 - gamma)**4) * (S**2 * A * np.log(c * S * A / delta)) / epsilon**2


gamma_slider = widgets.FloatSlider(value=gamma, min=0.0, max=1.0, step=0.01, description='Gamma:')
c_slider = widgets.IntSlider(value=c, min=1, max=10, step=1, description='C:')
delta_slider = widgets.FloatSlider(value=delta, min=0.0, max=1.0, step=0.01, description='Delta:')
epsilon_slider = widgets.FloatSlider(value=epsilon, min=0.0, max=1.0/(1- gamma), step=0.01, description='Epsilon:')

# Function to update N based on slider values
def update_N(gamma, c, delta, epsilon):
    N = (gamma / (1 - gamma)**4) * (S**2 * A * np.log(c * S * A / delta)) / epsilon**2
    print(f"Number of samples N: {N}")

# Display the sliders and link them to the update function
ui = widgets.VBox([gamma_slider, c_slider, delta_slider, epsilon_slider])
out = widgets.interactive_output(update_N, {'gamma': gamma_slider, 'c': c_slider, 'delta': delta_slider, 'epsilon': epsilon_slider})

display(ui, out)

Output()

* $\epsilon$を増やすと、$N$は減ります。上界は$\epsilon$で記述されているので、これが大きいと許容精度が高くなるので、それを達成するための$N$は少なくなります。
* $\gamma$を増やすと$N$は増えます。

### 命題2.1の証明

#### 補題2.2 (Simulation Lemma)

$$
Q^\pi-\widehat{Q}^\pi=\gamma\left(I-\gamma \widehat{P}^\pi\right)^{-1}(P-\widehat{P}) V^\pi
$$


#### 補題2.2の証明

$Q^\pi=\left(I-\gamma P^\pi\right)^{-1} r$を用いると、

$$
\begin{aligned}
Q^\pi-\widehat{Q}^\pi = & \left(I-\gamma P^\pi\right)^{-1} r-\left(I-\gamma \hat{P}^\pi\right)^{-1} r \\
= & \left(I-\left(I-\gamma P^\pi\right)\left(I-\gamma \hat{P}^\pi\right)^{-1}\right)\left(I-\gamma P^\pi\right)^{-1} r \\
= & \left(I-\gamma \hat{P}^\pi\right)^{-1}\left((I-\gamma \hat{P}^\pi)-\left(I-\gamma P^\pi\right)\right)\left(I-\gamma P^\pi\right)^{-1} r \\
\\
& Q^\pi=\left(I-\gamma P^\pi\right)^{-1} r より \\
\\
= & \left(I-\gamma \hat{P}^\pi\right)^{-1} \gamma\left(P^\pi-\hat{P}^\pi\right) Q^\pi \\
\\
& P^\pi Q^\pi=P V^\pi \\
& \hat{P}^\pi Q^\pi=\hat{P} V^\piより \\
\\
= & \gamma\left(I-\gamma \hat{P}^\pi\right)^{-1}(P-\hat{P}) V^\pi
\end{aligned}
$$

#### 補題2.3

任意の方策$\pi$, MDP $M$、およびベクトル$v \in \mathbb{R}^{|\mathcal{S}| \times|\mathcal{A}|}$に対して、
$$\left\|\left(I-\gamma P^\pi\right)^{-1} v\right\|_{\infty} \leq\|v\|_{\infty} /(1-\gamma)$$

が成り立ちます。


#### 補題2.3の証明
$$
\begin{aligned}
& v=\left(I-\gamma P^\pi\right)\left(I-\gamma P^\pi\right)^{-1}v \\
& ここで\left(I-\gamma P^\pi\right)^{-1} v=w とおくと、 \\
& v=(I-\gamma P^\pi) w \\
\end{aligned}
$$

$$
\begin{aligned}
\|v\|_{\infty} & =\left\|\left(I-\gamma P^\pi\right) \omega\right\|_{\infty} \\
& \geqq\|\omega\|_{\infty}-\gamma\| P^\pi \omega \|_{\infty} \\
\\
&\left\|P^\pi\omega\right\|_{\infty} \leq\|\omega\|_{\infty}より、\\
\\
& \geqq\|\omega\|_{\infty}-\gamma\left\| \omega\right\|_{\infty}\\
\\
& 並び変えると、\\
\\
&\frac{\|v\|_{\infty}}{1-\gamma} \geqq\left\|\left(I-\gamma P^\pi\right)^{-1} v\right\|_{\infty}\\

\end{aligned}
$$


#### 定理2.1(モデル精度)の証明

以下は$A.8$：ヘフディングの不等式を示しています。
$$
\begin{aligned}

& \operatorname{Pr}\left(\|\hat{q}-\vec{q}\|_1 \geqq \sqrt{d}\left(\frac{1}{\sqrt{N}}+\epsilon\right)\right) \leqq e^{-N\epsilon^2}\\
& \operatorname{Pr}\left(\|\hat{q}-\vec{q}\|_1 \geqq \sqrt{d}\left(\frac{1}{\sqrt{N}}+\epsilon\right)\right) \leqq \delta と おきます。 \\
\\
& 補集合を考えると、\\
\\
& \operatorname{Pr}\left(\|\hat{q}-\vec{q}\|<\sqrt{d}\left(\frac{1}{\sqrt{N}}+\epsilon\right)\right)>1-\delta \\
\\
&\deltaから\epsilonを導出します。 \\

& \delta=e^{-N \epsilon^2} \\
& \log (\delta)=-N \epsilon^2 となるので、\\
& \epsilon^2=-\frac{\log (\delta)}{N} \\
& \epsilon=\sqrt{\frac{\log \left(\frac{1}{\delta}\right)}{N}} となります。\\

&次に、\sqrt{d}\left(\frac{1}{\sqrt{N}}+\epsilon\right)を考えます。\\

& \sqrt{d}\left(\frac{1}{\sqrt{N}}+\epsilon\right)=\sqrt{d}\left(\frac{1}{\sqrt{N}}+\frac{\sqrt{\log \left(\frac{1}{\delta}\right)}}{\sqrt{N}}\right) \\
& =\sqrt{d}\left(\frac{1+\sqrt{\log \left(\frac{1}{\delta}\right)}}{\sqrt{N}}\right) \\
& cを適当におくと、 \\
& =c \frac{ \sqrt{d \log \left(\frac{1}{\delta}\right)}}{\sqrt{N}}\\


& 最後に\hat{q}、\vec{q}を状態遷移確率に置き換え、dを状態の次元数、Nをサンプル数mに置き換えれば証明は完了です。

\end{aligned}
$$

#### 定理2.1(一様値精度)の証明


$$
\begin{aligned}
\\
& 補題2.2より、\\
\\
& \left\|Q^\pi-\hat{Q}^\pi\right\|_{\infty} =\left\|\gamma\left(I-\gamma P^\pi\right)^{-1}(P-\hat{P}) V^\pi\right\|_{\infty} \\
\\
& 補題2.3より、\\
\\
& \leqq \frac{\gamma}{1-\gamma}\left\|(P-\hat{P}) V^\pi\right\|_{\infty} \\
\\
& 最大値ノルムを\text{max}で書き換えます。 \\
\\
& \leqq \frac{\gamma}{1-\gamma}\left(\max _{s, a}\|P(\cdot \mid s, a)-\hat{P}(\cdot \mid s, a )\|\right)\left\|V^\pi\right\|_{\infty} \\
\\
& 0 \leqq\left\|V^\pi\right\|_{\infty} \leq \frac{1}{1-\gamma}より、 \\
\\
& \leqq \frac{\gamma}{(1-\gamma)^2} \max _{\text {s,a }} \| P(\cdot\mid s, a)-\hat{P}\left(\cdot \mid s, a )\|\right. \\
\\
&定理2.1のモデル精度より、\\
\\
& \leqq \frac{\gamma}{(1-\gamma)^2} \cdot(1-\gamma)^{2} \epsilon \\

&\leqq \gamma \epsilon \\
\\
& 0 \leq \gamma < 1 より\\
\\
& \leq \epsilon \\
となります。
\end{aligned}
$$


#### TODO 定理3番目の証明


### 割引ありの場合における推定精度の定理

#### 定理2.6


$\delta \geq 0$、および適切に選択された絶対定数$c$に対して、次のことが成り立つ。

- （値推定）確率$1-\delta$で、

$$
\left\|Q^{\star}-\widehat{Q}^{\star}\right\|_{\infty} \leq \gamma \sqrt{\frac{c}{(1-\gamma)^3} \frac{\log (c|\mathcal{S}||\mathcal{A}| / \delta)}{N}}+\frac{c \gamma}{(1-\gamma)^3} \frac{\log (c|\mathcal{S}||\mathcal{A}| / \delta)}{N} .
$$

- （サブオプティマリティー）$N \geq \frac{1}{(1-\gamma)^2}$の場合、確率$1-\delta$で、

$$
\left\|Q^{\star}-Q^{\hat{\pi}^{\star}}\right\|_{\infty} \leq \gamma \sqrt{\frac{c}{(1-\gamma)^3} \frac{\log (c|\mathcal{S}||\mathcal{A}| / \delta)}{N}} .
$$

#### 定理2.6(値推定)の証明

補題2.2と2.3より、


$$
Q^\pi - \widehat{Q}^\pi = \gamma (I - \gamma \widehat{P}^\pi)^{-1} (P - \widehat{P})V^\pi
$$

から

$$
\|Q^\pi - \widehat{Q}^\pi\|_\infty \leq \frac{\gamma}{1 - \gamma} \|(P - \widehat{P}) V^\pi\|_\infty
$$

となります。

上記の不等式を最適方策 $\pi^\star$ と $\hat{\pi}^\star$ に適用すると、

$$
\begin{aligned}
\|Q^{\star} - \widehat{Q}^{\star}\|_\infty &\leq \frac{\gamma}{1 - \gamma} \|(P - \widehat{P}) V^{\star}\|_\infty, \\
\|Q^{\star} - \widehat{Q}^{\hat{\pi}^\star}\|_\infty &\leq \frac{\gamma}{1 - \gamma} \|(P - \widehat{P}) V^{\hat{\pi}^\star}\|_\infty
\end{aligned}
$$

となります。

分散補題2.10(**TODO：書く**)から、上式の上界における右辺にさらに上界を与えます。

、$\delta > 0$ とすると、確率 $1 - \delta$ で、

$$
|(P - \widehat{P}) V^{\star}| \leq \sqrt{\frac{2 \log (2 |\mathcal{S}| |\mathcal{A}| / \delta)}{N}} \sqrt{\text{Var}_P (V^{\star})} + \frac{1}{1 - \gamma} \frac{2 \log (2 |\mathcal{S}| |\mathcal{A}| / \delta)}{3N} \mathbb{1}
$$

が成り立ちます。同様に、確率 $1 - \delta$ で、

$$
|(P - \widehat{P}) V^{\hat{\pi}^\star}| \leq \sqrt{\frac{2 \log (2 |\mathcal{S}| |\mathcal{A}| / \delta)}{N}} \sqrt{\text{Var}_P (V^{\hat{\pi}^\star})} + \frac{1}{1 - \gamma} \frac{2 \log (2 |\mathcal{S}| |\mathcal{A}| / \delta)}{3N} \mathbb{1}
$$

が成り立ちます。

補題2.13(**TODO：書く**)より、確率 $1 - \delta$ で、


$$
\begin{aligned}
\text{Var}_P (V^{\star}) &\leq 2 \text{Var}_{\widehat{P}} (\widehat{V}^{\pi^{\star}}) + \Delta_{\delta, N}' \mathbb{1}, \\
\text{Var}_P (V^{\hat{\pi}^\star}) &\leq 2 \text{Var}_{\widehat{P}} (\widehat{V}^{\star}) + \Delta_{\delta, N}' \mathbb{1}
\end{aligned}
$$

が成り立ちます。なお、

$$
\Delta_{\delta, N}' := \frac{1}{(1 - \gamma)^2} \sqrt{\frac{18 \log (6 |\mathcal{S}| |\mathcal{A}| / \delta)}{N}} + \frac{1}{(1 - \gamma)^4} \frac{4 \log (6 |\mathcal{S}| |\mathcal{A}| / \delta)}{N}
$$

です。これらの不等式を用いて、 $\text{Var}_P (V^{\star})$ と $\text{Var}_P (V^{\hat{\pi}^\star})$ を置き換えます。

おきかえると、確率 $1 - \delta$ で、

$$
\begin{aligned}
&|(P - \widehat{P}) V^{\star}| \leq c \sqrt{\frac{\text{Var}_{\widehat{P}} (\widehat{V}^{\pi^{\star}}) \log (c |\mathcal{S}| |\mathcal{A}| / \delta)}{N}} + \Delta_{\delta, N}'' \mathbb{1}, \\
&|(P - \widehat{P}) V^{\hat{\pi}^\star}| \leq c \sqrt{\frac{\text{Var}_{\widehat{P}} (\widehat{V}^{\star}) \log (c |\mathcal{S}| |\mathcal{A}| / \delta)}{N}} + \Delta_{\delta, N}'' \mathbb{1}
\end{aligned}
$$

ただし、

$$
\Delta_{\delta, N}'' := c \frac{1}{1 - \gamma} \left( \frac{\log (c |\mathcal{S}| |\mathcal{A}| / \delta)}{N} \right)^{3/4} + \frac{c}{(1 - \gamma)^2} \frac{\log (c |\mathcal{S}| |\mathcal{A}| / \delta)}{N}
$$

となります。

**TODOこの後を書く**



##### 下界についての考え方
* 定理2.8を考えます。これは、ある条件を満たした任意のアルゴリズムに対するサンプル複雑度の下界を示しています。つまりどれだけ良いアルゴリズムであってもこれより少ないサンプル数で$\left\|Q^{\star}-\widehat{Q}^{\star}\right\|_{\infty}$が$\epsilon$以下になることはありません。これは生成モデルやいろいろな方法をふくめたアルゴリズムに対して成り立ちます。では次に定理2.6を見てみましょう。これは生成モデルを使って割引ありと有限設定でMDPを解いた場合の$\left\|Q^{\star}-\widehat{Q}^{\star}\right\|_{\infty}$の上界を示しています。重要なのはここからで、これを用いると系2.7を導きます。これは、割引ありの場合に生成モデルを用いた場合のアルゴリズムのサンプル複雑度の下界を示しています。なんとこれ定理2.8と同じ下界です。つまり、生成モデルをもちいた手法は割引設定において、どのアルゴリズムよりも良いか同等です。

###### \epsilonについて
* 最後の部分、$\epsilon$を達成するための～